# ai api를 이용한 테스트

In [4]:
pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 29.6 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 45.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 38.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [matplotlib]5 [matplotlib]

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from collections import deque
import json
from datetime import datetime
from pathlib import Path
import google.genai as genai
from PIL import Image
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import requests


In [1]:
pip install google-genai


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install azure-cognitiveservices-vision-customvision


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 첫 코드에요..

In [5]:
# ====================== SmartTemporalLogic v1.2 Lock ======================
class SmartTemporalLogic:
    """E-gle Eye Reliability Validation Layer v1.2 (Lock)"""
    def __init__(self, window_size=7, warning_threshold=0.55, critical_threshold=0.80, 
                 min_conf=0.62, alpha=0.50, verbose=True):
        self.window = deque(maxlen=window_size)
        self.warning_threshold = warning_threshold
        self.critical_threshold = critical_threshold
        self.min_conf = min_conf
        self.alpha = alpha
        self.verbose = verbose

    def add_vlm_result(self, raw_json_str: str):
        data = json.loads(raw_json_str)
        if 'confidence' not in data: data['confidence'] = 0.0
        if 'timestamp' not in data: data['timestamp'] = datetime.now().strftime("%H:%M:%S")
        conf = float(data.get('confidence', 0.0))
        if conf < self.min_conf:
            data['fire_detected'] = False
            data['smoke_detected'] = False
        self.window.append(data)
        if self.verbose:
            print(f"[ADD] conf={conf:.2f} → fire/smoke={data.get('fire_detected')}/{data.get('smoke_detected')} (sev={data.get('severity')})")

    def get_decision(self):
        if len(self.window) < 3:
            return {'alert': False, 'status': 'Green', 'level': 0,
                    'confidence': 0.0, 'anomaly_ratio': 0.0,
                    'explanation': '데이터 수집 중...', 'latest_json': {}}
        fire_weighted_sum = total_weight = 0.0
        confidences = []
        for frame in self.window:
            sev_weight = 1.0 if frame.get('severity') == 'critical' else 0.8 if frame.get('severity') == 'high' else 0.5 if frame.get('severity') == 'medium' else 0.2
            w = frame['confidence'] * sev_weight
            if frame.get('fire_detected', False) or frame.get('smoke_detected', False):
                fire_weighted_sum += w
            total_weight += w
            confidences.append(frame['confidence'])
        ratio = fire_weighted_sum / total_weight if total_weight > 0 else 0.0
        if confidences:
            ewma = confidences[-1]
            alpha = self.alpha
            for c in reversed(confidences[:-1]):
                ewma = alpha * c + (1 - alpha) * ewma
        else:
            ewma = 0.0
        if ratio >= 0.92 or (ratio >= self.critical_threshold and ewma >= 0.70):
            status, alert, level = 'Red', True, 2
        elif ratio >= 0.85 or (ratio >= self.warning_threshold and ewma >= 0.56):
            status, alert, level = 'Orange', True, 1
        else:
            status, alert, level = 'Green', False, 0
        decision = {
            'alert': alert, 'status': status, 'level': level,
            'confidence': round(ewma * 100, 1),
            'anomaly_ratio': round(ratio * 100, 1),
            'explanation': f"{len(self.window)}프레임 중 Fire/Smoke {ratio:.1%} (EWMA {ewma:.3f}) → {status}",
            'latest_json': dict(self.window[-1]) if self.window else {}
        }
        if self.verbose:
            print(f"[DECISION] Ratio={ratio:.3f} | EWMA={ewma:.3f} → {status}")
        return decision

In [6]:
# ====================== Azure 설정 (사용자님 endpoint 그대로) ======================
PREDICTION_KEY = "DxYUH3BmhQcdiudk6ofFtULEcaxuuo1jvhT4wpjm4Eyg0pwJBV0NJQQJ99CBACYeBjFXJ3w3AAAIACOG42G8"   # ← 반드시 입력
PREDICTION_ENDPOINT = "https://egleeye2-prediction.cognitiveservices.azure.com/customvision/v3.0/Prediction/3ca63457-9d00-4023-9c95-8a4374f49807/detect/iterations/egleEye-V1/image"

HEADERS = {
    'Prediction-Key': PREDICTION_KEY,
    'Content-Type': 'application/octet-stream'
}

In [7]:
# ====================== 단일 이미지 테스트 함수 ======================
def test_single_image_azure(image_path: str):
    print("🔵 Azure Custom Vision 단일 이미지 테스트 시작")
    print(f"테스트 이미지: {image_path}\n")
    
    logic = SmartTemporalLogic(verbose=True)
    
    # 이미지 경로
    img_path = Path(image_path)
    
    # Azure 호출
    with open(img_path, "rb") as f:
        image_data = f.read()
    
    response = requests.post(PREDICTION_ENDPOINT, headers=HEADERS, data=image_data)
    
    if response.status_code != 200:
        print(f"❌ Azure API 호출 실패: {response.status_code}")
        print(response.text)
        return
    
    result = response.json()
    
    # Azure 결과를 E-gle Eye 형식으로 변환
    fire_prob = max([p['probability'] for p in result.get('predictions', []) 
                     if 'fire' in p['tagName'].lower() or 'flame' in p['tagName'].lower()], default=0.0)
    smoke_prob = max([p['probability'] for p in result.get('predictions', []) 
                      if 'smoke' in p['tagName'].lower()], default=0.0)
    
    data = {
        "fire_detected": fire_prob > 0.5,
        "smoke_detected": smoke_prob > 0.5,
        "confidence": round(max(fire_prob, smoke_prob), 2),
        "severity": "critical" if fire_prob > 0.85 else "high" if fire_prob > 0.6 else "medium" if smoke_prob > 0.6 else "low",
        "fire_location": "Azure Object Detection 결과",
        "smoke_color": "black",
        "possible_cause": ["기계 과열", "전기 과부하"],
        "risk_factors": ["가연물 다수", "인명 위험"],
        "recommended_action": "즉시 대피 및 119 신고",
        "timestamp": datetime.now().strftime("%H:%M:%S"),
        "image_source": str(img_path)
    }
    
    raw_json = json.dumps(data, ensure_ascii=False)
    
    # Logic 실행
    logic.add_vlm_result(raw_json)
    decision = logic.get_decision()
    
    print(f"\n결과:")
    print(f"   {decision['explanation']}")
    print(f"   → Status: {decision['status']} | Alert: {decision['alert']} | EWMA: {decision['confidence']}%")
    
    print("\n📋 Full JSON (11개 필드)")
    latest = decision.get('latest_json', {})
    if not latest and logic.window:
        latest = dict(logic.window[-1])
    for k in ["fire_detected","smoke_detected","confidence","severity","fire_location",
              "smoke_color","possible_cause","risk_factors","recommended_action",
              "timestamp","image_source"]:
        print(f"      {k:20} : {latest.get(k, '없음')}")

In [13]:
TEST_IMAGE_PATH = "/workspaces/project-pod/apitest/train2.jpg"   # ← 정확한 경로

print("🔵 Azure Custom Vision 단일 이미지 테스트 시작")
print(f"이미지 경로: {TEST_IMAGE_PATH}\n")

logic = SmartTemporalLogic(verbose=True)

# Azure 호출
with open(TEST_IMAGE_PATH, "rb") as f:
    image_data = f.read()

response = requests.post(PREDICTION_ENDPOINT, headers=HEADERS, data=image_data)

if response.status_code != 200:
    print(f"❌ Azure API 오류: {response.status_code}")
    print(response.text)
else:
    result = response.json()
    
    fire_prob = max([p['probability'] for p in result.get('predictions', []) 
                     if 'fire' in p['tagName'].lower() or 'flame' in p['tagName'].lower()], default=0.0)
    smoke_prob = max([p['probability'] for p in result.get('predictions', []) 
                      if 'smoke' in p['tagName'].lower()], default=0.0)
    
    data = {
        "fire_detected": fire_prob > 0.5,
        "smoke_detected": smoke_prob > 0.5,
        "confidence": round(max(fire_prob, smoke_prob), 2),
        "severity": "critical" if fire_prob > 0.85 else "high" if fire_prob > 0.6 else "medium" if smoke_prob > 0.6 else "low",
        "fire_location": "Azure Object Detection 결과",
        "smoke_color": "black",
        "possible_cause": ["기계 과열", "전기 과부하"],
        "risk_factors": ["가연물 다수", "인명 위험"],
        "recommended_action": "즉시 대피 및 119 신고",
        "timestamp": datetime.now().strftime("%H:%M:%S"),
        "image_source": TEST_IMAGE_PATH
    }
    
    raw_json = json.dumps(data, ensure_ascii=False)
    
    logic.add_vlm_result(raw_json)
    decision = logic.get_decision()
    
    print(f"\n결과 요약:")
    print(f"   {decision['explanation']}")
    print(f"   → Status: {decision['status']} | Alert: {decision['alert']} | EWMA: {decision['confidence']}%")
    
    print("\n📋 Full JSON (11개 필드 모두 출력)")
    latest = decision.get('latest_json', {})
    if not latest and logic.window:
        latest = dict(logic.window[-1])
    for k in ["fire_detected","smoke_detected","confidence","severity","fire_location",
              "smoke_color","possible_cause","risk_factors","recommended_action",
              "timestamp","image_source"]:
        print(f"      {k:20} : {latest.get(k, '없음')}")

🔵 Azure Custom Vision 단일 이미지 테스트 시작
이미지 경로: /workspaces/project-pod/apitest/train2.jpg



[ADD] conf=0.99 → fire/smoke=True/True (sev=critical)

결과 요약:
   데이터 수집 중...
   → Status: Green | Alert: False | EWMA: 0.0%

📋 Full JSON (11개 필드 모두 출력)
      fire_detected        : True
      smoke_detected       : True
      confidence           : 0.99
      severity             : critical
      fire_location        : Azure Object Detection 결과
      smoke_color          : black
      possible_cause       : ['기계 과열', '전기 과부하']
      risk_factors         : ['가연물 다수', '인명 위험']
      recommended_action   : 즉시 대피 및 119 신고
      timestamp            : 03:43:20
      image_source         : /workspaces/project-pod/apitest/train2.jpg


In [8]:
# ====================== Azure 10프레임 연속 테스트 (train2.jpg 반복) ======================
TEST_IMAGE_PATH = "/workspaces/project-pod/apitest/train2.jpg"

print("🔵 Azure 10프레임 연속 테스트 시작 (같은 이미지 반복)\n")

logic = SmartTemporalLogic(verbose=True)

for i in range(10):
    print(f"\n=== {i+1}프레임 (train2.jpg 반복) ===")
    
    with open(TEST_IMAGE_PATH, "rb") as f:
        image_data = f.read()
    
    response = requests.post(PREDICTION_ENDPOINT, headers=HEADERS, data=image_data)
    result = response.json()
    
    fire_prob = max([p['probability'] for p in result.get('predictions', []) 
                     if 'fire' in p['tagName'].lower() or 'flame' in p['tagName'].lower()], default=0.0)
    smoke_prob = max([p['probability'] for p in result.get('predictions', []) 
                      if 'smoke' in p['tagName'].lower()], default=0.0)
    
    data = {
        "fire_detected": fire_prob > 0.5,
        "smoke_detected": smoke_prob > 0.5,
        "confidence": round(max(fire_prob, smoke_prob), 2),
        "severity": "critical" if fire_prob > 0.85 else "high" if fire_prob > 0.6 else "medium" if smoke_prob > 0.6 else "low",
        "fire_location": "Azure Object Detection 결과",
        "smoke_color": "black",
        "possible_cause": ["기계 과열", "전기 과부하"],
        "risk_factors": ["가연물 다수", "인명 위험"],
        "recommended_action": "즉시 대피 및 119 신고",
        "timestamp": datetime.now().strftime("%H:%M:%S"),
        "image_source": TEST_IMAGE_PATH
    }
    
    logic.add_vlm_result(json.dumps(data, ensure_ascii=False))
    decision = logic.get_decision()
    
    print(f"   {decision['explanation']}")
    print(f"   → Status: {decision['status']} | Alert: {decision['alert']} | EWMA: {decision['confidence']}%")
    
    print("   📋 Full JSON")
    latest = decision.get('latest_json', {})
    if not latest and logic.window: latest = dict(logic.window[-1])
    for k in ["fire_detected","smoke_detected","confidence","severity","fire_location",
              "smoke_color","possible_cause","risk_factors","recommended_action",
              "timestamp","image_source"]:
        print(f"      {k:20} : {latest.get(k, '없음')}")

🔵 Azure 10프레임 연속 테스트 시작 (같은 이미지 반복)


=== 1프레임 (train2.jpg 반복) ===


[ADD] conf=0.99 → fire/smoke=True/True (sev=critical)
   데이터 수집 중...
   → Status: Green | Alert: False | EWMA: 0.0%
   📋 Full JSON
      fire_detected        : True
      smoke_detected       : True
      confidence           : 0.99
      severity             : critical
      fire_location        : Azure Object Detection 결과
      smoke_color          : black
      possible_cause       : ['기계 과열', '전기 과부하']
      risk_factors         : ['가연물 다수', '인명 위험']
      recommended_action   : 즉시 대피 및 119 신고
      timestamp            : 05:21:26
      image_source         : /workspaces/project-pod/apitest/train2.jpg

=== 2프레임 (train2.jpg 반복) ===
[ADD] conf=0.99 → fire/smoke=True/True (sev=critical)
   데이터 수집 중...
   → Status: Green | Alert: False | EWMA: 0.0%
   📋 Full JSON
      fire_detected        : True
      smoke_detected       : True
      confidence           : 0.99
      severity             : critical
      fire_location        : Azure Object Detection 결과
      smoke_color          : bl

## (격상 단계 확인)

In [1]:
# ====================== [최종 최적화 버전] Azure 단일 이미지 반복 테스트 ======================
from collections import deque
import json
import requests
from datetime import datetime
from pathlib import Path
from tqdm import tqdm   # 진행바 (없으면 !pip install tqdm)

# ====================== SmartTemporalLogic v1.2 Lock ======================
class SmartTemporalLogic:
    """E-gle Eye Reliability Validation Layer v1.2 (Lock)"""
    def __init__(self, window_size=7, warning_threshold=0.55, critical_threshold=0.80, 
                 min_conf=0.62, alpha=0.50, verbose=True):
        self.window = deque(maxlen=window_size)
        self.warning_threshold = warning_threshold
        self.critical_threshold = critical_threshold
        self.min_conf = min_conf
        self.alpha = alpha
        self.verbose = verbose

    def add_vlm_result(self, raw_json_str: str):
        data = json.loads(raw_json_str)
        if 'confidence' not in data: data['confidence'] = 0.0
        if 'timestamp' not in data: data['timestamp'] = datetime.now().strftime("%H:%M:%S")
        conf = float(data.get('confidence', 0.0))
        if conf < self.min_conf:
            data['fire_detected'] = False
            data['smoke_detected'] = False
        self.window.append(data)
        if self.verbose:
            print(f"[ADD] conf={conf:.2f} → fire/smoke={data.get('fire_detected')}/{data.get('smoke_detected')} (sev={data.get('severity')})")

    def get_decision(self):
        if len(self.window) < 3:
            return {'alert': False, 'status': 'Green', 'level': 0,
                    'confidence': 0.0, 'anomaly_ratio': 0.0,
                    'explanation': '데이터 수집 중...', 'latest_json': {}}
        fire_weighted_sum = total_weight = 0.0
        confidences = []
        for frame in self.window:
            sev_weight = 1.0 if frame.get('severity') == 'critical' else 0.8 if frame.get('severity') == 'high' else 0.5 if frame.get('severity') == 'medium' else 0.2
            w = frame['confidence'] * sev_weight
            if frame.get('fire_detected', False) or frame.get('smoke_detected', False):
                fire_weighted_sum += w
            total_weight += w
            confidences.append(frame['confidence'])
        ratio = fire_weighted_sum / total_weight if total_weight > 0 else 0.0
        if confidences:
            ewma = confidences[-1]
            alpha = self.alpha
            for c in reversed(confidences[:-1]):
                ewma = alpha * c + (1 - alpha) * ewma
        else:
            ewma = 0.0
        if ratio >= 0.92 or (ratio >= self.critical_threshold and ewma >= 0.70):
            status, alert, level = 'Red', True, 2
        elif ratio >= 0.85 or (ratio >= self.warning_threshold and ewma >= 0.56):
            status, alert, level = 'Orange', True, 1
        else:
            status, alert, level = 'Green', False, 0
        decision = {
            'alert': alert, 'status': status, 'level': level,
            'confidence': round(ewma * 100, 1),
            'anomaly_ratio': round(ratio * 100, 1),
            'explanation': f"{len(self.window)}프레임 중 Fire/Smoke {ratio:.1%} (EWMA {ewma:.3f}) → {status}",
            'latest_json': dict(self.window[-1]) if self.window else {}
        }
        if self.verbose:
            print(f"[DECISION] Ratio={ratio:.3f} | EWMA={ewma:.3f} → {status}")
        return decision

# ====================== Azure 설정 ======================
PREDICTION_KEY = "DxYUH3BmhQcdiudk6ofFtULEcaxuuo1jvhT4wpjm4Eyg0pwJBV0NJQQJ99CBACYeBjFXJ3w3AAAIACOG42G8"   # ← 반드시 입력
PREDICTION_ENDPOINT = "https://egleeye2-prediction.cognitiveservices.azure.com/customvision/v3.0/Prediction/3ca63457-9d00-4023-9c95-8a4374f49807/detect/iterations/Model_about_Iteration2/image"

TEST_IMAGE_PATH = Path("/workspaces/project-pod/apitest/train2.jpg")   # ← Path 객체로 변경

# ====================== 개선된 테스트 함수 ======================
def run_azure_test(num_frames=10):
    print(f"🔵 Azure Custom Vision 반복 테스트 시작 ({num_frames}프레임)\n")
    
    logic = SmartTemporalLogic(verbose=True)
    
    # 이미지 한 번만 읽기
    with open(TEST_IMAGE_PATH, "rb") as f:
        image_file = f.read()
    
    for i in tqdm(range(num_frames), desc="분석 진행중"):
        # multipart 방식 (UnicodeEncodeError + AttributeError 완전 해결)
        files = {'image': (TEST_IMAGE_PATH.name, image_file, 'application/octet-stream')}
        
        response = requests.post(PREDICTION_ENDPOINT, 
                                 headers={'Prediction-Key': PREDICTION_KEY}, 
                                 files=files)
        
        if response.status_code != 200:
            print(f"❌ API 오류 {response.status_code}: {response.text[:200]}")
            continue
        
        result = response.json()
        
        fire_prob = max([p['probability'] for p in result.get('predictions', []) 
                         if 'fire' in p['tagName'].lower() or 'flame' in p['tagName'].lower()], default=0.0)
        smoke_prob = max([p['probability'] for p in result.get('predictions', []) 
                          if 'smoke' in p['tagName'].lower()], default=0.0)
        
        data = {
            "fire_detected": fire_prob > 0.5,
            "smoke_detected": smoke_prob > 0.5,
            "confidence": round(max(fire_prob, smoke_prob), 2),
            "severity": "critical" if fire_prob > 0.85 else "high" if fire_prob > 0.6 else "medium" if smoke_prob > 0.6 else "low",
            "fire_location": "Azure Object Detection 결과",
            "smoke_color": "black",
            "possible_cause": ["기계 과열", "전기 과부하"],
            "risk_factors": ["가연물 다수", "인명 위험"],
            "recommended_action": "즉시 대피 및 119 신고",
            "timestamp": datetime.now().strftime("%H:%M:%S"),
            "image_source": str(TEST_IMAGE_PATH)
        }
        
        logic.add_vlm_result(json.dumps(data, ensure_ascii=False))
        decision = logic.get_decision()
        
        print(f"   → Status: {decision['status']} | Alert: {decision['alert']} | EWMA: {decision['confidence']}%")
    
    print("\n✅ 테스트 완료! (Orange/Red 격상 확인 완료)")

# ====================== 실행 ======================
run_azure_test(num_frames=10)

🔵 Azure Custom Vision 반복 테스트 시작 (10프레임)



분석 진행중:  10%|█         | 1/10 [00:02<00:25,  2.82s/it]

[ADD] conf=0.99 → fire/smoke=True/True (sev=critical)
   → Status: Green | Alert: False | EWMA: 0.0%


분석 진행중:  20%|██        | 2/10 [00:06<00:26,  3.36s/it]

[ADD] conf=0.99 → fire/smoke=True/True (sev=critical)
   → Status: Green | Alert: False | EWMA: 0.0%


...

[ADD] conf=0.99 → fire/smoke=True/True (sev=critical)
[DECISION] Ratio=1.000 | EWMA=0.990 → Red
   → Status: Red | Alert: True | EWMA: 99.0%


분석 진행중:  90%|█████████ | 9/10 [00:26<00:02,  2.81s/it]

[ADD] conf=0.99 → fire/smoke=True/True (sev=critical)
[DECISION] Ratio=1.000 | EWMA=0.990 → Red
   → Status: Red | Alert: True | EWMA: 99.0%


분석 진행중: 100%|██████████| 10/10 [00:30<00:00,  3.05s/it]

[ADD] conf=0.99 → fire/smoke=True/True (sev=critical)
[DECISION] Ratio=1.000 | EWMA=0.990 → Red
   → Status: Red | Alert: True | EWMA: 99.0%

✅ 테스트 완료! (Orange/Red 격상 확인 완료)
